### Рыбы браконьеры) 


Делаю импорт

In [1]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
import matplotlib.pyplot as plt
from random import shuffle
from tensorflow.keras.applications import vgg16
import os
print(tf.__version__)

2.5.0


Проверяю , что нужная видеокарта на месте

In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17131794473011908830,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14272167936
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10091603592293498129
 physical_device_desc: "device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:08:00.0, compute capability: 8.6",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10084089856
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11347436779160658351
 physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:09:00.0, compute capability: 8.6"]

Задаю категории

In [3]:
class_names = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 
               'OTHER', 'SHARK', 'YFT']


Получаю список файлов

In [4]:
TRAIN_PREFIX = 'C:/Users/Doom/Downloads/the-nature-conservancy-fisheries-monitoring/train/train'
images_by_cat = []
for cat in class_names:
    folder1_files = []
    for root, dirs, files in os.walk(f"{TRAIN_PREFIX}/{cat}" , topdown=False):
        for i in files:
            folder1_files.append(i)
    print(len(folder1_files))
    images_by_cat.append(folder1_files)
    


975
200
117
67
465
299
176
734


Загружаю картинки 

In [5]:
target_size=(1200, 750)
X = []
y = []
train = []
IMG_SIZE = (1200, 750)
cat = 0 
for _ in images_by_cat:
    for __ in _:
        path = f'{TRAIN_PREFIX}/{class_names[cat]}/{__}'
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is not None:
            img_resized = cv2.resize(img, target_size)
            img_file = vgg16.preprocess_input(img_resized.astype(np.float32))
            img_arr = np.asarray(img_file)
            img_arr = img_arr / 255.0 
            X.append(img_arr)
            y.append(cat)
    cat += 1
X = np.asarray(X) # Keras only accepts data as numpy arrays 
y = np.asarray(y)        
        
print(X.shape)       

(3033, 750, 1200, 3)


In [6]:
X.shape

(3033, 750, 1200, 3)

In [7]:
print(y)

[0 0 0 ... 7 7 7]


In [8]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1200, 750, 3)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [9]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(X, y, epochs=5)

Epoch 1/5
95/95 [==============================] - 13s 110ms/step - loss: 80.4268 - accuracy: 0.5981
Epoch 2/5
95/95 [==============================] - 10s 106ms/step - loss: 18.4352 - accuracy: 0.6983
Epoch 3/5
95/95 [==============================] - 10s 106ms/step - loss: 11.0849 - accuracy: 0.6957
Epoch 4/5
95/95 [==============================] - 10s 106ms/step - loss: 5.9216 - accuracy: 0.7254
Epoch 5/5
95/95 [==============================] - 10s 104ms/step - loss: 8.7368 - accuracy: 0.7277


In [11]:
TEST_PREFIX = 'C:/Users/Doom/Downloads/the-nature-conservancy-fisheries-monitoring/test_stg1/test_stg1'
folder1_files = []
for root, dirs, files in os.walk(f"{TEST_PREFIX}" , topdown=False):
    for i in files:
        folder1_files.append(i)
print(len(folder1_files))



1000


In [12]:
X = []
y = []
print(folder1_files)

['img_00005.jpg', 'img_00007.jpg', 'img_00009.jpg', 'img_00018.jpg', 'img_00027.jpg', 'img_00030.jpg', 'img_00040.jpg', 'img_00046.jpg', 'img_00053.jpg', 'img_00071.jpg', 'img_00075.jpg', 'img_00102.jpg', 'img_00103.jpg', 'img_00109.jpg', 'img_00119.jpg', 'img_00120.jpg', 'img_00125.jpg', 'img_00128.jpg', 'img_00129.jpg', 'img_00133.jpg', 'img_00138.jpg', 'img_00141.jpg', 'img_00152.jpg', 'img_00161.jpg', 'img_00164.jpg', 'img_00170.jpg', 'img_00171.jpg', 'img_00172.jpg', 'img_00175.jpg', 'img_00180.jpg', 'img_00196.jpg', 'img_00212.jpg', 'img_00223.jpg', 'img_00230.jpg', 'img_00232.jpg', 'img_00244.jpg', 'img_00272.jpg', 'img_00280.jpg', 'img_00282.jpg', 'img_00285.jpg', 'img_00289.jpg', 'img_00292.jpg', 'img_00302.jpg', 'img_00311.jpg', 'img_00320.jpg', 'img_00331.jpg', 'img_00358.jpg', 'img_00374.jpg', 'img_00375.jpg', 'img_00376.jpg', 'img_00380.jpg', 'img_00383.jpg', 'img_00398.jpg', 'img_00407.jpg', 'img_00417.jpg', 'img_00432.jpg', 'img_00442.jpg', 'img_00446.jpg', 'img_00469.jp

In [13]:
target_size=(1200, 750)
predictions = []
cat = 0 
for __ in folder1_files:
    path = f'{TEST_PREFIX}/{__}'
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is not None:
        img_resized = cv2.resize(img, target_size)
        img_file = vgg16.preprocess_input(img_resized.astype(np.float32))
        img_arr = np.asarray(img_file)
        img_arr = img_arr / 255.0 
        prediction = model.predict(np.asarray(img_arr))
        predictions.append(img_arr)
predictions = np.asarray(predictions) # Keras only accepts data as numpy arrays        


ValueError: in user code:

    E:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1569 predict_function  *
        return step_function(self, iterator)
    E:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    E:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    E:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    E:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    E:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1552 run_step 